<a href="https://colab.research.google.com/github/dagousket/ML-course-VIB-2020/blob/master/Histone_Marks_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Let's go Deep Learning style!


In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import PIL
import tensorflow as tf

random_seed = 666
np.random.seed(random_seed)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

### Import and format data

In [2]:
train = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_test.csv")

# Save labels in external variable
train_labels = train.pop("Label")
train_index_col = train.pop("GeneId")
test_index_col = test.pop("GeneId")

In [3]:
marks = list(set([c_name.split("_")[0] for c_name in train.columns]))

# Shape of array will be 100 bins * 5 histone marks

def pool_marks(mybins, marks) :
  df = pd.DataFrame()
  totalread = 0.001
  for mark in marks :
      df.reset_index()
      df[mark] = mybins[[mark + "_" + str(bin) for bin in range(0,100)]].to_numpy()
      totalread = totalread + sum(df[mark])
  df = df.to_numpy() / totalread
  return df

def get_input(X, marks):
  res_out = X.apply(lambda y : pool_marks(y, marks), axis = 1)
  return pd.DataFrame(res_out)

X = [A for A in get_input(train, marks).iloc[:,0]]
X = np.array([np.array(dp) for dp in X])

T = [A for A in get_input(test, marks).iloc[:,0]]
T = np.array([np.array(dp) for dp in T])

In [19]:
X.shape

(10436, 100, 5)

In [35]:
#one-hot encode target column
from keras.utils import to_categorical
y_train = to_categorical(train_labels)
y_train = np.array([Y for Y in y_train])
print(y_train.shape)
y_train[0]

(10436, 2)


array([0., 1.], dtype=float32)

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_train, test_size=0.85)

### Make a DL model

In [40]:
from keras.layers.recurrent import LSTM
num_classes = 2

model = Sequential([
  layers.InputLayer(input_shape=(100,5)),
  layers.Conv1D(30, kernel_size=3, padding="valid", activation='relu', kernel_initializer='random_uniform', data_format ="channels_last"),
  layers.MaxPooling1D(pool_size=5, strides=1, padding='valid'),
  layers.Dropout(0.2),
  layers.TimeDistributed(layers.Dense(128, activation='relu')),
  layers.Bidirectional(LSTM(128, dropout=0.1, recurrent_dropout=0.1, return_sequences=True)),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(20, activation='relu'),
  layers.Dropout(0.4),
  layers.Dense(num_classes, activation='softmax')])


model.compile(optimizer='adam',
              loss='BinaryCrossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_9 (Conv1D)            (None, 98, 30)            480       
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 94, 30)            0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 94, 30)            0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 94, 128)           3968      
_________________________________________________________________
bidirectional_7 (Bidirection (None, 94, 256)           263168    
_________________________________________________________________
dropout_22 (Dropout)         (None, 94, 256)           0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 24064)            

In [41]:

history=model.fit(X_train, y_train, batch_size=100 , epochs=100 , verbose=1, validation_data=(X_test, y_test))


Epoch 1/100
16/16 [==============================] - 14s 872ms/step - loss: 0.6943 - accuracy: 0.4984 - val_loss: 0.6922 - val_accuracy: 0.8134
Epoch 2/100
16/16 [==============================] - 13s 836ms/step - loss: 0.6892 - accuracy: 0.5361 - val_loss: 0.6813 - val_accuracy: 0.5033
Epoch 3/100
16/16 [==============================] - 13s 835ms/step - loss: 0.6912 - accuracy: 0.5527 - val_loss: 0.6666 - val_accuracy: 0.7534
Epoch 4/100
16/16 [==============================] - 13s 836ms/step - loss: 0.6151 - accuracy: 0.7131 - val_loss: 0.4490 - val_accuracy: 0.8303
Epoch 5/100
16/16 [==============================] - 13s 844ms/step - loss: 0.5345 - accuracy: 0.7610 - val_loss: 0.5134 - val_accuracy: 0.8094
Epoch 6/100
16/16 [==============================] - 13s 832ms/step - loss: 0.5130 - accuracy: 0.7866 - val_loss: 0.4520 - val_accuracy: 0.8185
Epoch 7/100
16/16 [==============================] - 13s 820ms/step - loss: 0.5053 - accuracy: 0.7955 - val_loss: 0.4404 - val_accuracy:

In [44]:
from sklearn.metrics import log_loss, accuracy_score

predictions_train_prob = model.predict(X_train)
predictions_val_prob = model.predict(X_test)

print("Log-loss: (%f) %f"%(log_loss(y_train,predictions_train_prob[:,1]),log_loss(y_test,predictions_val_prob[:,1])))

In [ ]:
# Predict Kaggle set
predictions_test_prob = model.predict(X_test)

predictions_df = pd.DataFrame({"GeneId":X_test.index,"Label":predictions[:,1]})
predictions_df.to_csv('submission_rf.csv', index=False)